# Identifying Ideal Districts in Malaysia for Establishing New Gym Branches Using K-means Clustering

## link to nbviewer version of this notebook

[Link]()

## Webscraping Wikipedia page to get the district names in Peninsular Malaysia

### Importing libraries

In [2]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

print('Libraries imported.')

Libraries imported.


### Using BeautifulSoup to extract data

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_districts_in_Malaysia'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'lxml')
W=soup.find_all("a")

S = pd.Series(W) 
R= S.to_frame()
P=R[130:426]
P.columns =['District Name']

E=P['District Name'].astype(str).str.strip('[|]')

L= E.to_frame()

df1 = L[~L['District Name'].str.contains('new')]

df1 = df1[~df1['District Name'].str.contains('Council')]

df1 = df1[df1['District Name'].str.contains('District')]

G=df1['District Name'].astype(str)

W = G.str.split('title="',expand=True)

J=W[1]

D= J.str.split('>',expand=True)

Q=D[1]

K=Q.astype(str).str.strip("District</a")

K.columns=['Districts']

KO= K.to_frame()

KO.columns=['Dist']

KO1=KO.drop([209], axis = 0)

KO2=KO1.drop([324], axis = 0)

KO3=KO2.drop([414], axis = 0)

KO3=KO3.drop([405], axis = 0)

KO3=KO3.drop([351], axis = 0)

df2 = pd.DataFrame({"Dist":["Larut", "Dungun"]}) 

KOO=KO3.append(df2)

KOO.to_csv('KOO_2.csv', index=False) # The dataframe is saved as a csv file. 

KOO.head() # The dataframe containing the extracted district names



,Dist
130,Batu Pahat
134,Johor Bahru
139,Kluang
143,Kota Tinggi
147,Kulai


## Getting latitude and longitude values for districts in KOO dataframe

In [5]:
latitude4=[]
longitude4=[]
address1=[]
for district in KOO['Dist']:
    address = '{}, Malaysia'.format(district)
    address1.append(address)

    geolocator = Nominatim(user_agent="District")
    location2 = geolocator.geocode(address)
    
    latitude5 = location2.latitude
    latitude4.append(latitude5)
    
    longitude5 = location2.longitude
    longitude4.append(longitude5)

In [6]:
S1 = pd.Series(latitude4) 
R1= S1.to_frame()

S2 = pd.Series(longitude4) 
R2= S2.to_frame()

R1.columns =['Latitude']
R2.columns =['Longitude']


In [7]:
R1.head()

,Latitude
0,1.908342
1,1.495304
2,2.069914
3,1.688710
4,1.671807


In [8]:
R2.head()

,Longitude
0,102.995623
1,103.755084
2,103.381181
3,103.964021
4,103.561497


In [9]:
R1.to_csv('Latitude_R1.csv', index=False)
R2.to_csv('Longitude1_R2.csv', index=False)

### The district names, longitude and latitude values , mean and median income and population data were combined into one file named district_2.csv file

In [10]:
DF = pd.read_csv(r'C:\Users\sofia\Documents\Jupyter\district_2.csv')
DF.head()

,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
0,Batu Pahat,1.908342,102.995623,417458,5516,6277,NaN,NaN,3,4,2
1,Johor Bahru,1.495304,103.755084,1386569,6518,8198,NaN,NaN,3,4,2
2,Kluang,2.069914,103.381181,298332,5652,6928,NaN,NaN,3,4,2
3,Kota Tinggi,1.688710,103.964021,193210,5652,6928,NaN,NaN,3,4,2
4,Kulai,1.671807,103.561497,251650,6114,6901,NaN,NaN,3,4,2


#### The number of shopping malls and number of gym/fitness centre will be filled up after getting  the data from the Foursquare API

## Map of the districts on Peninsular Malaysia

In [11]:
map = folium.Map(location=[3.974, 102.4381], zoom_start=7)

# markers to map
for lat, lng, district in zip(DF['Latitude'], DF['Longitude'], DF['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 

In [12]:
map

## Using Foursquare API to get total number of shopping malls and gyms

In [13]:
CLIENT_ID = 'IMIOYQCOC3FOJ0DBOZLRR2G0NYIFQP2K42PWCT5FPWPYQHBT' 
CLIENT_SECRET = '5PPMFVGRSKPARKKZUT3SCYIJX1BJRGZQLACUX50XQMYEHMCC' 
VERSION = '20180605' 
LIMIT = 100 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IMIOYQCOC3FOJ0DBOZLRR2G0NYIFQP2K42PWCT5FPWPYQHBT
CLIENT_SECRET:5PPMFVGRSKPARKKZUT3SCYIJX1BJRGZQLACUX50XQMYEHMCC


### Getting total number of gyms in each district

In [14]:
def getVenues(names, latitudes, longitudes, radius=30000):  
    global Num4
    venue_name=[]
    total=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d176941735&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
           
        
        results = requests.get(url).json()["response"]['venues']
        
                 
        Q1 = pd.Series(results)
        Q2= Q1.to_frame()
        Q2.columns=['Venue']
        Q3=Q2['Venue'].astype(str)
        Q7 = Q3.str.split(',',expand=True)
        index1 = Q2.index
        number_of_rows = len(index1) 
        AS=total.append(number_of_rows)
        print("The number of gym in {} is {}".format(name,number_of_rows))
        XZ=Q7[1]

        XZ1 = pd.Series(XZ) 
        CV= XZ1.str.split(':',expand=True)
        O=CV[1]
        AW=venue_name.append(O)
        print(O)
        
    
    Num4= pd.DataFrame(data={'Total':total})
    
    return()
    
        
        

In [15]:
tv = getVenues(names=DF['District'],
                                   latitudes=DF['Latitude'],
                                   longitudes=DF['Longitude']
                                  )

The number of gym in Batu Pahat  is 39
0                  '7 FITNESS & DANCE'
1                 'Gold Fitness World'
2            'MY GYM Excercise Centre'
3              '37 Image Solution Gym'
4                         'S.Work GYM'
5                  'Hillview Loft Gym'
6          'Pusat Kegiatan Rakan Muda'
7                                'Gym'
8                           'Diva Gym'
9                           'J11 Club'
10      'ShaufikBoss Fitness Gym Room'
11               'Rainbow Fitness Gym'
12                         'Rambo Gym'
13                         'KKTSN Gym'
14                    'Home Gym @ Ria'
15                           'Gym Man'
16                   'Rambo Gymnasium'
17                               'gym'
18                           'Fit Gym'
19                      'Katerina Gym'
20                   'Underground Gym'
21                       'Rainbow Gym'
22                     'Mini Gym KKTF'
23                    'U.N.I.Q.U.E Gym
24              'Vigor Fi

The number of gym in Mersing  is 7
0                                     'JM YB'
1                                       'Gym'
2     'SuperHeroes Gymnasium & Fitness Center
3                           'Mersing Fitness'
4                            'FBR Teluk Buih'
5                            'Gym 1 Malaysia'
6     'SuperHeroes Gymnasium & Fitness Centre
Name: 1, dtype: object
The number of gym in Muar  is 39
0                      'Energym'
1            '7 FITNESS & DANCE'
2             'Triangle Fitness'
3          'Rainbow Fitness Gym'
4                      'GYM ude'
5              'Underground Gym'
6             'Weird Lady Space'
7      'Atas Katil Peluk Bantal'
8                     'J11 Club'
9            'Bunny Bubbly Boom'
10               'aerobic class'
11                    'Diva Gym'
12                   'Rambo Gym'
13          'Tihani Gossip Area'
14                    'Skipping'
15             'Rambo Gymnasium'
16                      'MY GYm'
17               'Divas Fi

The number of gym in Kota Setar  is 49
0               'Optimus Gym And Fitness Centre'
1                              '111 Fitness Gym'
2                                     'Iron Gym'
3                                'Aesthetic Gym'
4                                      'Gym Utc'
5                    'MuscleWorks World Fitness'
6                        'Athlete Fitness Studio
7                    'GN Fitness TamanGurunJaya'
8                          'Lady Dreams Fitness'
9            'Friendly Muscle Fitness a.k.a 贵宾楼'
10                                   'Alora Gym'
11                         'Amir120 Fitness GYM'
12                                    'iron gym'
13                                 'Tri Fitness'
14                       'LZ Lady Fitness N Gym'
15                                  'Gym Rakyat'
16                    'fisioterapi putih seroja'
17     'Just Fitness Gymnasium & Fitness Centre'
18                     'Friendly Muscle Fitness'
19                            

The number of gym in Langkawi  is 26
0                      'Gimnasium Rakyat (LADA)'
1                             'G9 Gym & Fitness'
2                         'Rizal Fitness Centre'
3                              'The Health Club'
4                                     'Adiprene'
5                                 'private club'
6                               'Gymnasium AKMAL
7                     'eagle Gymnasium (Yaacob)'
8                    'Gym @ Four Seasons Resort'
9                         'Berjaya Langkawi Gym'
10                                   'Gymnasium'
11                'Gym @ Langkawi Seaview Hotel'
12                           'Gym @ Tanjung Rhu'
13                  'RIZAL GYM & FITNESS CENTRE'
14            'Fitness - Resorts World Langkawi'
15                               'Gym@The Danna'
16     'Gymnaium @ Century Suria Service Suites'
17                                   'V Fitness'
18       'Leisure Center Meritus Pelangi Resort'
19                          'bay

The number of gym in Yan  is 49
0                   'the legend gym & fitness center'
1                                          'Iron Gym'
2                              'Empire Gym & Fitness'
3                    'Optimus Gym And Fitness Centre'
4                                  'Rush Fitness Gym'
5                                   '111 Fitness Gym'
6                                  'Everyone Fitness'
7                                    'Silver Fitness'
8                                       'one fitness'
9                         'MuscleWorks World Fitness'
10                        'Toilet Mahsuri UiTM Kedah'
11                                       'Paksi Gyml'
12                               'Mini Gym Kg Masjid'
13                                   'Gym Reban Ayam'
14                     'Pusat Latihan Tomoi Abg Hery'
15                                        'Titan Gym'
16                                      'Tatical Gym'
17     "judy's professional beauty treatment acade

...

The number of gym in Kerian  is 45
0                            'Trufit Fitness'
1                            'Delone Fitness'
2                  'Red Gym & Fitness Centre'
3                             'Young Fitness'
4                             'Altera B3 Gym'
5            "D'city Gym Kerian Sentral Mall"
6           'ART MUSCLE GYM & FITNESS CENTRE'
7                                'Gym rakyat'
8                                        'Ti'
9                   'Power Master Gym Centre'
10                          'gym kumuniti BS'
11                             'Cik Lym Gym*'
12     'bathroom @ absolute fitness autocity'
13                               'DiamondGym'
14                              'Fitness Ace'
15                         'Body & Works Gym'
16                                  'CRC gym'
17                                 'Sado Gym'
18                          'CRC TAIPING GYM'
19                                  'Mini Gym
20              'Gimnasium Rakyat 1 Malaysia'

The number of gym in Muallim  is 18
0               'Pusat Gimnasium Sin Ban Seng'
1          'Pusat Latihan Zumba Sekiah Makmur'
2                                     'C2 Gym'
3                                     'my gym'
4                            'Workout Station 
5      'Bilik Bebanan 2 Gymnasium Pusat Sukan'
6                                'Gym ligamas'
7                              'El dorado gym'
8                                  'Pro^2-Gym'
9                       'Gymnasium Rakyat KKB'
10                               'Lite Fitness
11                           'EverlyFit Studio
12           'Gym Latihan Zumba Sekiah Makmur'
13                        'Ezy Gym (JRD Land)'
14                               'Monster gym'
15                            'Ezy gym b.kali'
16                       'Ladies gym & beauty'
17                   'Gym Proton Sports Centre
Name: 1, dtype: object
The number of gym in Perak Tengah  is 50
0                                'Frank Fitness Centre

The number of gym in Hulu Selangor  is 49
0                               'The Best Gym'
1               'Gym Room Perdana Residence 2'
2                          'World Fitness Gym'
3                   'Rockafella Gym & Fitness'
4                    'Sri Damansara Clubhouse'
5                               'HERCULES GYM'
6                               'Valencia Gym'
7        'Pro Hardcore Gym And Fitness Centre'
8             'First World Fitness Gym Center'
9                                    'Ezy Gym'
10                              'Fitness Zone'
11                  'Astana Putra Condominium'
12                                'Sunway Gym'
13         'Pusat Latihan Zumba Sekiah Makmur'
14                          'Titanium Merikel'
15                                   'Gym rat'
16                           'Workout Station 
17                        'Gym@desa Aman Puri'
18     'Bilik Bebanan 2 Gymnasium Pusat Sukan'
19                            'ezy gym rawang'
20                

The number of gym in Petaling  is 50
0                 'Petalz Residence Gym Room'
1         'Body Factory Gym & Fitness Center'
2                         'Celebrity Fitness'
3           'Anytime Fitness TTDI Plaza Vads'
4                    'Fitness First Platinum'
5                             'Gofit Fitness'
6                         'Ezy Gym & Fitness'
7                                 'BSP21 Gym'
8                                'Uptown Gym'
9                     'Anytime Fitness Nexis'
10               'Anytime Fitness @8 Kinrara'
11                           'Chi-X Starling'
12                        'Celebrity Fitness'
13                             'Gym@The Zest'
14                   'The Club @ Bukit Utama'
15            'Anytime Fitness @Gateway Mall'
16         'Menara Maybank Recreation Centre'
17                'Celebrity Fitness Express'
18                        'Celebrity Fitness'
19            'Perdana Exclusive Condominium'
20                        'Celebrity Fitnes

The number of gym in Marang  is 34
0                      'Gimnasium Rakyat Marang'
1      'Tanjong Vista Gym @ Tanjong Vista Hotel'
2                   'TH Ultimate California Gym'
3                            'Top Superstar Gym'
4                                     'quno gym'
5                                      'TER GYM'
6                            'MHI Gym & Fitness'
7                                     'LeGo Gym'
8                                      'Faq Gym'
9                        'Gym 1 Malaysia Marang'
10                                  'Robbie Gym'
11                          'Gym Gong Tok Nasek'
12                      'Gymnasium Permai Hotel'
13                  'Gym Cengal Condo Clubhouse'
14                                'muscular gym'
15                                 'Dumbell Gym'
16                                  'Gym Rakyat'
17                                  'Katang Gym'
18              'UMT Gymnasium @ Kompleks Sukan'
19                                

### Getting total number of shopping malls

In [16]:
def getVenues1(names, latitudes, longitudes, radius=30000):  
    global Num5
    venue_name=[]
    total=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d1fd941735&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
           
        
        results = requests.get(url).json()["response"]['venues']
        
                 
        Q1 = pd.Series(results)
        Q2= Q1.to_frame()
        Q2.columns=['Venue']
        Q3=Q2['Venue'].astype(str)
        Q7 = Q3.str.split(',',expand=True)
        index1 = Q2.index
        number_of_rows = len(index1) 
        AS=total.append(number_of_rows)
        print("The number of shopping malls in {} is {}".format(name,number_of_rows))
        XZ=Q7[1]

        XZ1 = pd.Series(XZ) 
        CV= XZ1.str.split(':',expand=True)
        O=CV[1]
        AW=venue_name.append(O)
        print(O)
        
    
    Num5= pd.DataFrame(data={'Total':total})
    
    return()
    
        

In [18]:
tv1 = getVenues1(names=DF['District'],
                                   latitudes=DF['Latitude'],
                                   longitudes=DF['Longitude']
                                  )

The number of shopping malls in Batu Pahat  is 47
0                                'Tesco'
1                      'Batu Pahat Mall'
2                            'The Store'
3                    'Pasaraya Wellward'
4                           'Square One'
5                           'The Summit'
6                       'Old Street 老街坊'
7                 'EconSave @ Yong Peng'
8                               'Summit'
9                    'City-Link Express'
10                        'Butik summit'
11                           'BCB plaza'
12            'U0 Superstore Batu Pahat'
13                               'UO BP'
14                  'fotocharlie bpmall'
15               'Lampu 3 colour summit'
16                         'ma minimart'
17              'kedai runcit cik sakin'
18                   'Pasaraya UO Summit
19                      'Toilet library'
20           'Pasar Mini Sentiasa Bagus'
21                        'Nirwana mall'
22                         'AEON Rawang'
23     

The number of shopping malls in Kulai  is 50
0                                      'Tesco'
1                      'Johor Premium Outlets'
2                                    'The Mall
3                              'Plaza Angsana'
4                              'Paradigm Mall'
5                    'Beletime Danga Bay Mall'
6                                      'Tesco'
7             'AEON Kulaijaya Shopping Centre'
8                           'Tesco Mall Kulai'
9                                   'AEON BIG'
10                                     'Mydin'
11                             'Holiday Plaza'
12                                     'Mydin'
13                                  'Econsave'
14                                 'The Store'
15          'AEON Tebrau City Shopping Centre'
16                                  'Econsave'
17                       'AEON Mall Kulaijaya'
18                         "D'Pristine Medini"
19                    'Toppen Shopping Centre'
20          'AE

The number of shopping malls in Tangkak  is 48
0                     'Wetex Parade Shopping Centre'
1                                        'U Sentral'
2                        'Astaka Muar Shopping Mall'
3                                       'Nafas Mall'
4                        'One Segamat Shopping Mall'
5                           'Kini Mart Bukit Gambir'
6                         'Mydin Mall Jasin Bestari'
7                                   'K-Mall Tg Agas'
8                                  'KIP Mart Segamat
9                                      'Mydin Jasin'
10                            'Nirwana Super Market'
11                                       'The Store'
12                         'K-Mall Shopping Complex'
13                                          'Fresco'
14                                       'Guan Hong'
15                                   'UO Superstore'
16                                     'K-mart Muar'
17                                       'Beauberry'

The number of shopping malls in Kubang Pasu  is 48
0                                    'Giant Superstore'
1                                     'Kayangan Square'
2                                               'Tesco'
3                                      'Alor Star Mall'
4         'Pekan Rabu @ Ukir Mall (Kompleks Sementara)'
5                                        'Aman Central'
6                                       'Tesco Mergong'
7                                 'Kompleks Pekan Rabu'
8                            'Pasar Borong Jalan Putra'
9                                     'Souq Al-Bukhary'
10                                             'C Mart'
11                                         'City Plaza'
12           'Pacific Hypermarket & Departmental Store'
13                       'Kompleks Bazar Jejawi MPK Ps'
14                                         'Jitra Mall'
15                                       'C-Mart Jitra'
16                                             'C-Mar

The number of shopping malls in Pendang  is 49
0                                            'Tesco'
1                                      'Aneka Gurun'
2                                            'Tesco'
3                                   'Alor Star Mall'
4                                   'Central Square'
5      'Pekan Rabu @ Ukir Mall (Kompleks Sementara)'
6                                     'Aman Central'
7                                    'Tesco Mergong'
8                              'Kompleks Pekan Rabu'
9                         'Pasar Borong Jalan Putra'
10                               'Manjaku Baby Mall'
11                             'Billion Supermarket'
12                                     'Star Parade'
13                                          'C Mart'
14                                 'Souq Al-Bukhary'
15        'Pacific Hypermarket & Departmental Store'
16                         'Pasaraya Karnival Jerai'
17                                      'City Plaza'

...

The number of shopping malls in Kuala Kangsar  is 50
0              'HQ Giant hypermarket bercham'
1                               'Ipoh Parade'
2      'AEON Ipoh Station 18 Shopping Centre'
3                            'Aeon Kinta City
4                              'Taiping Mall'
5                    'AEON Mall Ipoh Klebang'
6                      'Mydin Gugusan Manjoi'
7        'AEON Midtown Falim Shopping Centre'
8                             'TF Value Mart'
9                                   'Parkson'
10               'Kinta City Shopping Centre'
11                              'Tesco Extra'
12                        'Econsave Jelapang'
13                        'Baskin Robbin Tpg'
14                'Mr DIY @ Tesco Station 18'
15                    'Econsave Angsana Mall'
16                        'Angsana Mall Ipoh'
17                                 'Econsave'
18                            'Festival Walk'
19                                    'Tesco'
20                         

The number of shopping malls in Hulu Langat  is 50
0                         'Cheras Leisure Mall'
1                    'Centrepoint Bandar Utama'
2                             'Setia City Mall'
3                     'Bangsar Shopping Centre'
4                         'IPC Shopping Centre'
5                               'Subang Parade'
6                                    'Jaya One'
7                         'Central i-City Mall'
8                      'MyTOWN Shopping Centre'
9                                  'Suria KLCC'
10                            'Setapak Central'
11                              'Paradigm Mall'
12                               'The Starling'
13                        'AEON Mall Shah Alam'
14                                  'The Curve'
15                   'Alamanda Shopping Centre'
16                        'Mid Valley Megamall'
17          'AEON Taman Maluri Shopping Centre'
18                     'Atria Shopping Gallery'
19         '1 Utama Shopping Centre (

The number of shopping malls in Kuala Selangor  is 50
0                                   'Setia City Mall'
1                               'Central i-City Mall'
2                                  'Tesco Setia Alam'
3                           'Kompleks PKNS Shah Alam'
4                'AEON Rawang Anggun Shopping Centre'
5                                             'Tesco'
6                   'AEON Bukit Raja Shopping Centre'
7                                   'Plaza Shah Alam'
8                                      'Klang Parade'
9                                       'Centro Mall'
10                               'Plaza Alam Sentral'
11                'Shah Alam City Centre (SACC Mall)'
12                                          'SB Mall'
13                       'Star Avenue Lifestyle Mall'
14                                   'wims marketing'
15                                         'Kip Mall'
16                                'Tesco Puncak Alam'
17                          

The number of shopping malls in Besut  is 29
0                       'Pasaraya Econjaya'
1                       'Giant Hypermarket'
2                      'Pasar Tani Kg Buluh
3                            'Pasar minggu'
4                                    'Tesco
5                   'al-ikhsan pasir puteh'
6                           'CHOP GIM HUAT'
7                          'Pasar Semerak~'
8                            'Dmart Chalok'
9                       'metropoint kajang'
10                                 'PLPPWP'
11           'Pasaraya Eco RM2 Kuala Besut'
12                   'ECONSAVE Pasir Puteh'
13            'Pasar Pagi Penarik (jumaat)'
14                          'Secret Recipe'
15                            'Machangster'
16                     'Shf hikmat trading'
17                            'Lua Trading'
18         'Perubatan Tradisional Mok Wook'
19     'Pasaraya Econjaya Lama Pasir Puteh'
20                             'Pasar Pagi'
21     'Gedung Pakaian Ten Ten 

The number of shopping malls in Setiu  is 16
0                'Pasaraya Econjaya'
1                'Giant Hypermarket'
2               'Pasar Tani Kg Buluh
3                 'kedai pok pendek'
4                     'Pasar minggu'
5                    'CHOP GIM HUAT'
6                     'Dmart Chalok'
7                'metropoint kajang'
8                           'PLPPWP'
9      'Pasar Pagi Penarik (jumaat)'
10                   'Secret Recipe'
11              'Shf hikmat trading'
12                     'Lua Trading'
13          'Wakaf Tengah Bike Shop'
14               'Giant Mall Jerteh'
15                  'kedai ayah wan'
Name: 1, dtype: object


## Adding the number of shopping malls and number of gym to the dataset. The final dataset is saved as Full.csv

In [19]:
DF['Number of Shopping Malls']=Num5
DF['Number of Gym/fitness centre']=Num4

In [21]:
DF.to_csv('Full.csv', index=False)

In [22]:
DF.head()

,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
0,Batu Pahat,1.908342,102.995623,417458,5516,6277,47,39,3,4,2
1,Johor Bahru,1.495304,103.755084,1386569,6518,8198,50,50,3,4,2
2,Kluang,2.069914,103.381181,298332,5652,6928,43,13,3,4,2
3,Kota Tinggi,1.688710,103.964021,193210,5652,6928,50,49,3,4,2
4,Kulai,1.671807,103.561497,251650,6114,6901,50,50,3,4,2


In [23]:
DF3 = pd.read_csv(r'C:\Users\sofia\Documents\Jupyter\Full.csv')

In [24]:
CF=DF3

## K-Means clustering with 7 clusters

In [25]:
kclusters = 7
from sklearn.preprocessing import StandardScaler
tc = CF.drop('District', 1)
tc1=StandardScaler().fit_transform(tc)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tc1)

kmeans.labels_[0:10]

array([4, 2, 4, 4, 4, 4, 4, 4, 4, 4])

In [26]:
CF.insert(0, 'Cluster Labels', kmeans.labels_)

CF.head() 

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
0,4,Batu Pahat,1.908342,102.995623,417458,5516,6277,47,39,3,4,2
1,2,Johor Bahru,1.495304,103.755084,1386569,6518,8198,50,50,3,4,2
2,4,Kluang,2.069914,103.381181,298332,5652,6928,43,13,3,4,2
3,4,Kota Tinggi,1.688710,103.964021,193210,5652,6928,50,49,3,4,2
4,4,Kulai,1.671807,103.561497,251650,6114,6901,50,50,3,4,2


### Cluster 0

In [27]:
CF.loc[CF['Cluster Labels'] == 0]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
10,0,Baling,5.672928,100.918204,135646,3811,4971,13,2,3,2,1
23,0,Gua Musang,4.879272,101.965233,90057,3079,4214,2,1,3,1,2
24,0,Jeli,5.695523,101.844937,40637,3079,4214,17,7,3,1,2
26,0,Kuala Krai,5.530017,102.199106,109461,3079,4214,21,8,3,1,2
30,0,Tanah Merah,5.766305,102.034148,121319,3079,4214,38,14,3,1,2
36,0,Jempol,2.850272,102.508196,116576,4579,5887,25,2,3,1,2
42,0,Bentong,3.520932,101.910618,119817,3979,5012,28,22,3,2,1
43,0,Bera,3.236674,102.541772,97882,3979,5012,17,17,3,2,1
45,0,Jerantut,3.936659,102.371955,91096,3979,5012,14,13,3,2,1
47,0,Lipis,4.324251,101.908585,89730,3979,5012,6,2,3,2,1


### Cluster 1

In [28]:
CF.loc[CF['Cluster Labels'] == 1]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
11,1,Bandar Baharu,5.131119,100.497278,42341,3811,4971,49,50,3,2,1
12,1,Kota Setar,6.122603,100.369032,366787,3811,4971,49,49,3,2,1
13,1,Kuala Muda,5.707315,100.495289,456605,3811,4971,50,50,3,2,1
14,1,Kubang Pasu,6.335638,100.372322,220740,3811,4971,48,49,3,2,1
15,1,Kulim,5.366377,100.552571,287694,3811,4971,50,50,3,2,1
16,1,Langkawi,6.370039,99.792863,94777,3811,4971,26,26,3,2,1
17,1,Padang Terap,6.256651,100.664580,62896,3811,4971,46,26,3,2,1
18,1,Pendang,5.972084,100.550185,94962,3811,4971,49,49,3,2,1
19,1,Pokok Sena,6.167313,100.517432,49506,3811,4971,48,49,3,2,1
20,1,Sik,5.818854,100.744277,67378,3811,4971,49,31,3,2,1


### Cluster 2

In [29]:
CF.loc[CF['Cluster Labels'] == 2]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
1,2,Johor Bahru,1.495304,103.755084,1386569,6518,8198,50,50,3,4,2
68,2,Gombak,3.233044,101.714574,682226,7903,9908,50,50,4,3,1
69,2,Hulu Langat,3.070825,101.763669,1156585,7851,9593,50,50,4,3,1
70,2,Hulu Selangor,3.536744,101.594935,198132,5421,9463,50,49,4,3,1
71,2,Klang,3.043125,101.449611,861189,6724,8606,50,50,4,3,1
73,2,Kuala Selangor,3.362102,101.345550,209590,7225,6532,50,50,4,3,1
74,2,Petaling,3.086134,101.664208,1812633,7904,10792,50,50,4,3,1
75,2,Sabak Bernam,3.687115,101.058059,105777,7225,9463,20,3,4,3,1
76,2,Sepang,2.800862,101.709401,211361,8174,10121,50,50,4,3,1


### Cluster 3

In [30]:
CF.loc[CF['Cluster Labels'] == 3]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
77,3,Besut,5.610237,102.506370,140952,4694,5776,29,12,1,3,2
78,3,Dungun,4.769677,103.416885,154932,4694,5776,27,20,1,3,2
79,3,Hulu Terengganu,4.910689,102.765070,72052,4694,5776,6,1,1,3,2
80,3,Kemaman,4.192789,103.119487,171383,5355,6565,16,12,1,3,2
81,3,Kuala Terengganu,5.329701,103.138313,343284,4694,5776,47,35,1,3,2
82,3,Marang,5.206546,103.204279,97857,4694,5776,47,34,1,3,2
83,3,Setiu,5.569059,102.763677,55517,4694,5776,16,11,1,3,2


### Cluster 4

In [31]:
CF.loc[CF['Cluster Labels'] == 4]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
0,4,Batu Pahat,1.908342,102.995623,417458,5516,6277,47,39,3,4,2
2,4,Kluang,2.069914,103.381181,298332,5652,6928,43,13,3,4,2
3,4,Kota Tinggi,1.688710,103.964021,193210,5652,6928,50,49,3,4,2
4,4,Kulai,1.671807,103.561497,251650,6114,6901,50,50,3,4,2
5,4,Mersing,2.355646,103.765113,70894,5652,6928,6,7,3,4,2
6,4,Muar,2.132507,102.784776,247957,5371,6928,45,39,3,4,2
7,4,Pontian,1.515380,103.474217,155541,5652,6928,50,49,3,4,2
8,4,Segamat,2.489551,102.972858,189820,5652,6928,39,13,3,4,2
9,4,Tangkak,2.284004,102.650964,136852,5652,6928,48,49,3,4,2


### Cluster 5

In [32]:
CF.loc[CF['Cluster Labels'] == 5]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
53,5,Central Seberang Perai,5.353979,100.457346,371975,5409,6771,50,50,3,4,5
54,5,North Seberang Perai,5.342175,100.427949,295979,5409,6771,50,50,3,4,5
55,5,Northeast Penang Island,5.378792,100.250172,520242,5409,6771,50,50,3,4,5
56,5,South Seberang Perai,5.340490,100.427137,171045,5409,6771,50,50,3,4,5
57,5,Southwest Penang Island,5.378792,100.250172,202142,5409,6771,50,50,3,4,5


### Cluster 6

In [33]:
CF.loc[CF['Cluster Labels'] == 6]

,Cluster Labels,District,Latitude,Longitude,Population,Median Income per month,Mean Income per month,Number of Shopping Malls,Number of Gym/fitness centre,1st most age group,2nd most age group,3rd most age group
32,6,Alor Gajah,2.383668,102.210469,182666,5588,6849,50,50,3,1,2
33,6,Central Melaka,2.224511,102.261466,503127,5877,7142,50,50,3,1,2
34,6,Jasin,2.305810,102.422339,135317,5588,6296,50,50,3,1,2
35,6,Jelebu,3.036626,102.117921,39200,4579,5887,44,45,3,1,2
37,6,Kuala Pilah,2.743076,102.215488,66092,4579,5887,50,48,3,1,2
38,6,Port Dickson,2.522894,101.794513,115361,4579,5887,50,48,3,1,2
39,6,Rembau,2.588869,102.094340,43011,4579,5887,50,48,3,1,2
40,6,Seremban,2.718788,101.940451,555935,4579,6621,50,49,3,1,2
41,6,Tampin,2.472154,102.232350,84889,4579,5887,50,49,3,1,2
46,6,Kuantan,3.798564,103.321990,461906,3979,5012,50,47,3,2,1


## Visualising Clusters on Map of Peninsular Malaysia

In [34]:

map_clusters = folium.Map(location=[3.9743, 102.4381], zoom_start=7)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(CF['Latitude'], CF['Longitude'], CF['District'], CF['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)


In [35]:
map_clusters